# ASSIGNMENT 2 - IDC 409
## CLASSIFYING MNIST DATASET BY ARTIFICIAL NEURAL NETWORKS INVOLVING -
* NO HIDDEN LAYER <BR>
* SINGLE HIDDEN LAYER <BR>
* TWO HIDDEN LAYERS <BR>
VIA THE FOLLOWING ACTIVATION FUNCTIONS - <BR>
* SIGMOID
* TANH
* RELU

In [1]:
import pandas as pd
import numpy as np
from math import sqrt 
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split

### Problem 1

In [2]:
def load_data(path):
    Y=[]
    X=[]
    #path='/Users/sanatmishra27/Downloads/mnist_train.csv'
    data = np.loadtxt(path,delimiter=",")
    for row in data:
        X.append(row[:-1])
        Y.append(int(row[-1]))
    return np.array(X),Y

data=load_data('/Users/sanatmishra27/Downloads/mnist_train.csv')


#### Before we move onto the rest of the problems, we do the following- 

The data is first preprocessed to ensure only non-zero pixel values for each of the 10,000 samples are contained so as to update weights without errors.<br>
Thus we shall scale down all values to a range of [0.01,1] by multiplying by a factor and adding 0.01 to remove zero valued entries that obstruct updating weights. <br>
Finally, we one-hot encode the class labels. <br>
Note that we have taken transposes of matrices appropriately at each step to ensure valid multiplication of matrices.

In [3]:
fac =  0.99/255
x_train = np.asfarray(data[0][:, 0:])*fac + 0.01

#x_train, x_test, y_train, y_test = train_test_split(X, data[1],test_size=0.3,random_state=0,stratify=data[1])
'''Stratify was used to split the data while testing such that it created training in the same proportion of 
labels as is present in the original data. '''

#One Hot Encoding the labels
n_values = int(np.max(data[1])+1)
y_train=np.eye(n_values)[data[1]]

### Problem 2

Now let's proceed to first initialize a weight matrix and a bias matrix. These two matrices are very critical to the performance of the neural network.

1) WEIGHT MATRIX

The weights have been initialised by constructing a matrix whose values are picked from a standard normal curve.

In [4]:
input=x_train.shape[1]
output=10
w= (np.random.randn(10,input))
w.shape

(10, 784)

2) BIAS MATRIX

The bias helps in controlling the value at which activation function (in this case the softmax function) will trigger. Thus it helps in quickening the search for the best-fit function.

In [5]:
b = np.random.randn(10, x_train.shape[0])  
b.shape

(10, 10000)

Now, compiling all the above information into a zero hidden layer neural network.

In [6]:
def feedforward(b,w):
    z = np.dot(w,x_train.T)+b
    return z   # z is a (10x7000) matrix

def softmax(s):
    exps = np.exp(s-np.max(s)) # We subtract the additional term to ensure values don't greatly shoot up.
    return exps/np.sum(exps, axis=0, keepdims=True)

def errordiff(pred, real):
    samplesize = real.shape[0]
    res = pred - real
    return res /samplesize

def cross_entropy(predict,real):
    samplesize = real.shape[0] #7,000
    L_sum = np.sum(np.dot(np.log(predict),real))
    L = -(1/samplesize) * L_sum
    return L

A few points to note in the above, the softmax function is subtracted by its maximum value to ensure that values don't become extraordinarily big. <br>
The error function returns the diffeence scaled by the sample size. 

In [7]:
x_train.shape

(10000, 784)

Finally, the function below computes and returns the updated weights and biases. <br>
The list of matrices used in the following code, along with their dimensions are as follows - <br>

1) x_train (10000x784) <br>
2) y_train (10000x10) <br>
3) delta (10000x10) <br>
4) w (10x784) <br>
5) b (10x10000)

We now code for the backprop. section and update the weights.

In [8]:
def finalbackprop(X,Y,w,b,lr):
    delta = errordiff(X.T, Y) 
    w -= lr * np.dot(delta.T,x_train) 
    b -= lr * np.sum(delta.T, axis=0, keepdims=True) 
    return w,b

X=softmax(feedforward(b,w)) # X is a (10x7000) matrix
Y=y_train  # Y is a (7000x10) matrix

results=finalbackprop(X,Y,w,b,0.001)
print('The updates weights are',results[0])
print('The updates biases are',results[1])


The updates weights are [[-0.89406232 -0.58249404 -1.3651342  ... -0.62042341 -0.86671015
  -0.1765741 ]
 [-1.64467576  1.48308452  1.89171623 ... -0.01678469  0.47459725
  -0.51772445]
 [-1.51521819 -0.8424277  -0.92399453 ...  0.37174272  1.05012878
   0.32129177]
 ...
 [ 0.72752567 -0.39740686 -1.29129193 ... -0.40021915 -0.52782602
   0.30886132]
 [-0.64961018 -0.31653074  1.58863764 ... -0.50497616 -0.97614895
  -0.58987728]
 [ 0.42453968 -0.47077215 -0.10170718 ... -0.67563836 -1.80365163
  -0.23404338]]
The updates biases are [[-1.53750397 -0.72439772  0.03050305 ...  0.49751233  0.50597019
  -0.56982844]
 [ 1.20582748  0.23832829  0.70772384 ... -0.28959508 -1.43657412
  -0.15519177]
 [ 1.17012055  0.35039404 -0.69412379 ... -0.17447291  0.53454542
   1.36785118]
 ...
 [-0.72156778 -1.59744124 -0.91651469 ...  0.45359374 -0.09664347
   0.58468342]
 [-1.31282311 -1.2114665  -1.69700858 ... -1.72157437 -1.17675753
  -0.73309454]
 [ 1.26131524  1.58322103 -0.0563262  ...  0.090838

### Problem 3

Now we code for the single hidden layer. By adjusting the value of N, we can control the number of neurons in the layer.

In [9]:
N = 20     # neurons for hidden layers
lr = 0.01  # learning rate

w1 = np.random.randn(x_train.shape[1], N) 
b1 = np.random.randn(1,N) 
w2 = np.random.randn(N, 10)
b2 = np.random.randn(1, 10)

w=[w1,w2]
b=[b1,b2]

Then, we define functions that we invoke along with the feedforward steps-

In [10]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def softmax(s):
    exps = np.exp(s-np.max(s)) 
    return exps/np.sum(exps, axis=1, keepdims=True)

def sigmoid_derv(x):
    return x * (1 - x)  

def errordiff(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def feedforward(x,w,b):
    z1 = np.dot(x,w[0]) + b[0]
    a1 = sigmoid(z1) 
    z2 = np.dot(a1,w[1]) + b[1]
    a2 = softmax(z2)  
    return a1,a2



Finlly, we code the backpropagation section, including returning updated weights and biases.

In [11]:
def backprop(a1,a2,w,b):
   
    a2_del = errordiff(a2, y_train) 
    z2_delta = np.dot(a2_del, w[1].T)  
    a1_del = z2_delta *sigmoid_derv(a1)   
                                                  
    w2 = w[1]- lr * np.dot(a1.T, a2_del)
    b2 = b[1] - lr * np.sum(a2_del, axis=0)
    
    w1 = w[0]- lr * np.dot(x_train.T, a1_del)  #X.T is (783x10000)
    b1 = b[0] - lr * np.sum(a1_del, axis=0)
    
    weights=[]
    biases=[]
    weights.extend([w1,w2])
    biases.extend([b1,b2])
    
    return weights,biases

X=feedforward(x_train,w,b) # X is a (10x7000) matrix
Y=y_train  # Y is a (7000x10) matrix

results=backprop(X[0],X[1],w,b)

print('The updates weights are',results[0])
print('The updates biases are',results[1])



The updates weights are [array([[-0.31252249,  0.09177449,  0.17649915, ..., -0.76956261,
         0.86953219,  0.08318242],
       [-2.38953646, -1.24360515,  0.6354934 , ...,  2.33701497,
         0.65364067, -0.52491783],
       [ 0.73230949,  0.02805841, -0.75257765, ...,  1.35439344,
         0.78540072, -1.10121413],
       ...,
       [ 0.13364363,  0.94423892, -0.01708292, ..., -1.13997884,
         0.37784275, -0.57784318],
       [ 0.88501366, -0.26987159, -0.14471453, ...,  0.34333959,
         1.57944757, -0.76996534],
       [-0.81291261, -0.74370468,  1.2644272 , ..., -0.78094806,
         0.63357681,  1.04501222]]), array([[-6.88260965e-01,  4.48675625e-01,  1.53502929e+00,
        -7.08049016e-02, -5.78452013e-01, -1.21458012e+00,
        -1.54421432e-01, -9.43979325e-01,  1.03728750e-01,
         1.96380982e-01],
       [ 1.86187298e+00,  1.31730479e+00, -3.04355811e-01,
        -9.93018940e-01, -1.32333798e+00,  1.92873247e+00,
         7.29098895e-02, -1.02233608e+00

### Problem 4

Problem 4, Problem 5 and Problem 6 are identical, with the latter two being built on this one. <br>
This two hidden layers are initialised in the following cell, along with the weight and bias matrices.

In [12]:
N = 10      # neurons for hidden layers
lr = 0.001  # learning rate
Y=y_train  

w1 = np.random.randn(x_train.shape[1], N) 
b1 = np.random.randn(1,N) 
w2 = np.random.randn(N, N)
b2 = np.random.randn(1, N)
w3=  np.random.randn(N, 10)
b3 = np.random.randn(1,10)

w=[w1,w2,w3]
b=[b1,b2,b3]

In [13]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def softmax(s):
    exps = np.exp(s-  np.max(s))
    return exps/np.sum(exps, axis=1, keepdims=True)

def sigmoid_derv(x):
    return (x) * (1 - x) 

def errordiff(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def feedforward(x,weights,biases):
    z1 = np.dot(x,weights[0]) + biases[0]
    a1 = sigmoid(z1) 
    
    z2 = np.dot(a1,weights[1]) + biases[1]  
    a2 = sigmoid(z2)  

    z3 = np.dot(a2, weights[2]) + biases[2]  
    a3 = softmax(z3) 
    
    return a1,a2,a3   

We now code for the backpropagation using the functions defined above. <br>
#### R is taken to be the number of rows in the input matrix, earlier it was taken to be 10000 at the start of the problem.

In [14]:
def backprop(a1,a2,a3,weights,biases):
   
    a3_del = errordiff(a3, y_train)             # a3_del is (Rx10 matrix)
    z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN matrix)
        
    a2_delta = z2_delta *sigmoid_derv(a2)     # a2_delta is (RxN)
    z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
        
    a1_del = z1_delta*sigmoid_derv(a1)      # a1_del is (RxN)
    
    w3 = weights[2]- lr * np.dot(a2.T, a3_del)                  #w3 is (Nx10)
    b3 = biases[2] - lr * np.sum(a3_del, axis=0, keepdims=True) #b3 is (1x10)
        
    w2 = weights[1]- lr * np.dot(a1.T,a2_delta)                   #w2 is (NxN)
    b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)  #b2 is (1x5)
        
    w1 = weights[0]- lr * np.dot(x_train.T, a1_del)                   #w1 is (784xN)
    b1 = biases[0]- lr * np.sum(a1_del, axis=0)                 #b1 is (1xN)
        
    weights=[]
    biases=[]
    weights.extend([w1,w2,w3])
    biases.extend([b1,b2,b3])
        
    return weights,biases


X=feedforward(x_train,w,b) # X is a (10x7000) matrix

results=backprop(X[0],X[1],X[2],w,b)
print('The updates weights are',results[0])
print('The updates biases are',results[1])


The updates weights are [array([[ 0.67407409,  0.3757721 ,  0.88690447, ..., -0.77641841,
         0.50804385, -0.9045933 ],
       [ 1.40062852, -0.51525304, -1.78851204, ...,  1.18770084,
         1.51110333, -1.21353052],
       [ 1.41437577,  1.19022806,  0.63476196, ...,  0.73935823,
         0.72608125, -0.79300185],
       ...,
       [-1.48310581, -0.87508251,  0.91985754, ..., -1.56455178,
         0.37041671,  1.0186037 ],
       [-1.53768137,  0.68274765,  0.36013257, ..., -0.18147091,
        -1.14677775, -1.06832061],
       [ 0.19359093,  1.61302044,  1.7220381 , ...,  0.88082665,
        -1.20215715, -0.66464174]]), array([[ 0.29627287,  0.21620461,  0.43517362,  0.18514882, -0.2430146 ,
        -1.94519513,  1.1697059 , -1.80090294,  1.70215221,  1.71394306],
       [ 0.60994204, -0.82047391,  0.83591908, -0.64554344,  0.01957567,
        -0.23470401, -0.10730442, -0.48145732, -0.46224903, -1.51170491],
       [ 1.15375297,  0.02431071, -1.07044002,  0.66762256, -1.3166

### Problem 5

The initialization remains identical to the previous case, a weight matrix 'w' consisting of three weight parameters and a bias matrix 'b' consisting of three bias parameters.

In [15]:
N = 20    # neurons for hidden layers
lr = 0.01 # learning rate

w1 = np.random.randn(x_train.shape[1], N) 
b1 = np.random.randn(1,N) 
w2 = np.random.randn(N, N)
b2 = np.random.randn(1, N) 
w3=  np.random.randn(N, 10)
b3 = np.random.randn(1,10)

w=[w1,w2,w3]
b=[b1,b2,b3]

Additional functions such as activation functions and derivatives of 'tanh','relu' and 'sigmoid' have been added.

In [16]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def tanh(s):
    return np.tanh(s)

def relu(s):      
    return np.maximum(0,s) 

def softmax(s):
    exps = np.exp(s-  np.max(s))
    return exps/np.sum(exps, axis=1, keepdims=True)

def tanhder(s):
    return (1-s**2)

def reluder(s):
    return 1.0*(s>0)

def sigmoid_derv(x):
    return x * (1 - x) 

def errordiff(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples


From here on, activation inputs and backpropagation are contained in a single function. <br>
Once the activation function type has been determined, the input is created by feeding through two steps of activation and one softmax. <br>
Then, differential values are calculated using chain rule that will aid in backpropagation. <br>
Finally, the updated weights are returned.

In [17]:
def NeuralNet(x,y,weights,biases,lr,activation):
    
    if activation == "sigmoid":   
        
        z1 = np.dot(x,weights[0]) + biases[0]     # z1 is (RxN)
        a1 = sigmoid(z1)                          # a1 is (RxN)
    
        z2 = np.dot(a1,weights[1]) + biases[1]    # z2 is (RxN)
        a2 = sigmoid(z2)                          # a2 is (RxN)

        z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
        a3 = softmax(z3)                          # a3 is (Rx10)

        
        a3_del = errordiff(a3, y_train)           # a3_del is (Rx10)
        z2_delta = np.dot(a3_del, weights[2].T)   # z2_delta is (RxN)
        
        a2_delta = z2_delta *sigmoid_derv(a2)     # a2_delta is (RxN)
        z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
        
        a1_del = z1_delta*sigmoid_derv(a1)        # a1_del is (RxN)
    
        w3 = weights[2]- lr * np.dot(a2.T, a3_del)                  #w3 is (Nx10)
        b3 = biases[2] - lr * np.sum(a3_del, axis=0, keepdims=True) #b3 is (1x10)
        
        w2 = weights[1]- lr * np.dot(a1.T,a2_delta)                   #w2 is (NxN)
        b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)  #b2 is (1x5)
        
        w1 = weights[0]- lr * np.dot(X.T, a1_del)                   #w1 is (784xN)
        b1 = biases[0]- lr * np.sum(a1_del, axis=0)                 #b1 is (1xN)
        
        weights=[]
        biases=[]
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        
        return weights,biases
    
    elif activation == "tanh":  
        
        z1 = np.dot(x, weights[0]) + biases[0]    # z1 is (RxN)
        a1 = tanh(z1)                             # a1 is (RxN)
        
        z2 = np.dot(a1, weights[1]) + biases[1]   # z2 is (RxN)
        a2 = tanh(z2)                             # a2 is (RxN)
        
        z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
        a3 = softmax(z3)                          # a3 is (Rx10)
        
        a3_del = errordiff(a3, y_train)             # a3_del is (Rx10)
        z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN)
        
        a2_delta = z2_delta *tanhder(a2)          # a2_delta is (RxN)
        z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
        
        a1_del = z1_delta*tanhder(a1)           # a1_del is (RxN)
        
        
        w3 = weights[2] - lr * np.dot(a2.T, a3_del)                 #w3 is (Nx10)
        b3 = biases[2]- lr * np.sum(a3_del, axis=0, keepdims=True)  #b3 is (1x10)
        
        w2 = weights[1] -lr * np.dot(a1.T,a2_delta)                   #w2 is (NxN)
        b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)  #b2 is (1x5)
        
        w1 = weights[0] -lr * np.dot(x.T, a1_del)                   #w1 is (784xN)
        b1 = biases[0]- lr * np.sum(a1_del, axis=0)                 #b1 is (1xN)
        
        weights=[]
        biases=[]
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        
        return weights,biases
    
    elif activation == "relu":      
        
        z1 = np.dot(x,weights[0]) + biases[0]     # z1 is (RxN)
        a1 = relu(z1)                             # a1 is (RxN) 
        
        z2 = np.dot(a1,weights[1]) + biases[1]    # z2 is (RxN)
        a2 = relu(z2)                             # a2 is (RxN) 
        
        z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
        a3 = softmax(z3)                          # a3 is (Rx10)
        
        
        a3_del = errordiff(a3, y_train)             # a3_del is (Rx10)
        z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN)
    
        a2_delta = z2_delta*reluder(a2)           # a2_delta is (RxN)
        z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
       
        a1_del = z1_delta*reluder(a1)           # a1_del is (RxN)
    
        
        w3 = weights[2] -lr * np.dot(a2.T, a3_del)                  #w3 is (Nx10)
        b3 = biases[2]- lr * np.sum(a3_del, axis=0, keepdims=True)  #b3 is (1x10)
        
        w2 = weights[1]- lr * np.dot(a1.T,a2_delta)                   #w2 is (NxN)
        b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)  #b2 is (1x5)
        
        w1 = weights[0]- lr * np.dot(x.T, a1_del)                   #w1 is (784xN)
        b1 = biases[0]- lr * np.sum(a1_del, axis=0)                 #b1 is (1xN)
        
        weights=[]
        biases=[]
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        
        return weights,biases

    
results=NeuralNet(x_train,y_train,w,b,lr,'relu')   
print('The updates weights are',results[0])
print('The updates biases are',results[1])

The updates weights are [array([[ 0.85576027,  0.27512277,  0.28360944, ...,  0.7919832 ,
        -1.2618036 ,  0.12917041],
       [ 0.07278122, -0.07555369,  0.02920275, ...,  1.42552701,
         0.64564324,  0.03746054],
       [-0.01230897,  0.54347978,  0.29565185, ...,  0.69903835,
         0.27678684, -1.33022671],
       ...,
       [-0.54596993,  1.10810127, -1.40998812, ..., -0.31291899,
         1.18476861,  0.32852765],
       [ 0.44568798, -2.63726733, -1.51127398, ..., -1.25564567,
         1.01032798,  0.38360471],
       [-1.47158202,  1.0667911 , -0.50143151, ..., -0.2640538 ,
        -0.93854619,  0.29067035]]), array([[-0.55272297, -0.091603  ,  0.40878732, -0.70904778,  0.49355514,
         1.13851314,  0.08785083, -0.15631827, -1.14589406,  0.07756585,
        -2.1526645 ,  0.12432614, -2.01753758, -0.83435512,  0.58439715,
        -0.25983306, -0.00776084, -0.79064996,  0.85458005,  1.16990901],
       [ 0.43669043, -1.14531951,  2.53850692,  2.21452024,  1.17376

### Problem 6

The problem is identical to the previous one with just a momentum parameter that has been added and an epoch value.

In [18]:
N = 20    # neurons for hidden layers
lr = 0.01 # learning rate

w1 = np.random.randn(x_train.shape[1], N) 
b1 = np.random.randn(1,N) 
w2 = np.random.randn(N, N)
b2 = np.random.randn(1, N) 
w3=  np.random.randn(N, 10)
b3 = np.random.randn(1,10)

w=[w1,w2,w3]
b=[b1,b2,b3]

In [19]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def tanh(s):
    return np.tanh(s)

def relu(s):      
    return np.maximum(0,s) 

def softmax(s):
    exps = np.exp(s-  np.max(s))
    return exps/np.sum(exps, axis=1, keepdims=True)

def tanhder(s):
    return (1-s**2)

def reluder(s):
    return 1.0*(s>0)

def sigmoid_derv(x):
    return x * (1 - x) 

def errordiff(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

  


In [20]:
def NeuralNet(x,y,weights,biases,lr,activation,momentum,mf,epochs):
    
    if activation == "sigmoid":   
        
        for i in range(epochs):
            
            z1 = np.dot(x,weights[0]) + biases[0]     # z1 is (RxN)
            a1 = sigmoid(z1)                          # a1 is (RxN)
    
            z2 = np.dot(a1,weights[1]) + biases[1]    # z2 is (RxN)
            a2 = sigmoid(z2)                          # a2 is (RxN)

            z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
            a3 = softmax(z3)                          # a3 is (Rx10)

        
            a3_del = errordiff(a3, y_train)             # a3_del is (Rx10)
            z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN)
        
            a2_delta = z2_delta *sigmoid_derv(a2)     # a2_delta is (RxN)
            z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
        
            a1_del = z1_delta*sigmoid_derv(a1)      # a1_del is (RxN)
        
            w3 = weights[2]- lr * np.dot(a2.T, a3_del)+ mf * momentum[0]  #checked
            b3 = biases[2] - lr * np.sum(a3_del, axis=0, keepdims=True)+ mf*momentum[3]  
        
            w2 = weights[1]- lr * np.dot(a1.T,a2_delta)+ mf * momentum[1]+mf*momentum[1]  
            b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)+ mf*momentum[4]  
        
            w1 = weights[0]- lr * np.dot(x.T, a1_del)+ mf * momentum[2]
            b1 = biases[0]- lr * np.sum(a1_del, axis=0)+ mf*momentum[5]  
            
            momentum=[]
            momentum.extend([- lr * np.dot(a2.T, a3_del),- lr * np.dot(a1.T,a2_delta),- lr * np.dot(x.T, a1_del),-lr*np.sum(a3_del, axis=0, keepdims=True),-lr*np.sum(a2_delta, axis=0, keepdims=True),-lr * np.sum(a1_del, axis=0)])
        
        weights=[]
        biases=[]
        
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        
        
        return weights,biases
    
    elif activation == "tanh":  
        
        for i in range(epochs):
        
            z1 = np.dot(x, weights[0]) + biases[0]    # z1 is (RxN)
            a1 = tanh(z1)                             # a1 is (RxN)
        
            z2 = np.dot(a1, weights[1]) + biases[1]   # z2 is (RxN)
            a2 = tanh(z2)                             # a2 is (RxN)
        
            z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
            a3 = softmax(z3)                          # a3 is (Rx10)
        
            a3_del = errordiff(a3, y_train)             # a3_del is (Rx10)
        
            z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN)
            a2_delta = z2_delta *tanhder(a2)          # a2_delta is (RxN)
        
            z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
            a1_del = z1_delta*tanhder(a1)           # a1_del is (RxN)
        
        
            w3 = weights[2] - lr * np.dot(a2.T, a3_del)+ mf * momentum[0]                #w3 is (Nx10)
            b3 = biases[2]- lr * np.sum(a3_del, axis=0, keepdims=True)+ mf * momentum[3] #b3 is (1x10)
        
            w2 = weights[1] -lr * np.dot(a1.T,a2_delta)+ mf * momentum[1]                  #w2 is (NxN)
            b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)+ mf * momentum[4] #b2 is (1x5)
        
            w1 = weights[0] -lr * np.dot(x.T, a1_del)+ mf * momentum[2]                  #w1 is (784xN)
            b1 = biases[0]- lr * np.sum(a1_del, axis=0)+ mf * momentum[5]                #b1 is (1xN)
            
            momentum=[]
            momentum.extend([- lr * np.dot(a2.T, a3_del),- lr * np.dot(a1.T,a2_delta),- lr * np.dot(x.T, a1_del),-lr*np.sum(a3_del, axis=0, keepdims=True),-lr*np.sum(a2_delta, axis=0, keepdims=True),-lr * np.sum(a1_del, axis=0)])
        
        
        weights=[]
        biases=[]
        
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        momentum.extend([- lr * np.dot(a2.T, a3_del),- lr * np.dot(a1.T,a2_delta),- lr * np.dot(x.T, a1_del),-lr*np.sum(a3_del, axis=0, keepdims=True),-lr*np.sum(a2_delta, axis=0, keepdims=True),-lr * np.sum(a1_del, axis=0)])
        
        return weights,biases
    
    elif activation == "relu":     
        
        for i in range(epochs):
        
            z1 = np.dot(x,weights[0]) + biases[0]     # z1 is (RxN)
            a1 = relu(z1)                             # a1 is (RxN) 
        
            z2 = np.dot(a1,weights[1]) + biases[1]    # z2 is (RxN)
            a2 = relu(z2)                             # a2 is (RxN) 
        
            z3 = np.dot(a2, weights[2]) + biases[2]   # z3 is (Rx10)
            a3 = softmax(z3)                          # a3 is (Rx10)
        
        
            a3_del = errordiff(a3, y_train)             # a3_del is (Rx10)
        
            z2_delta = np.dot(a3_del, weights[2].T) # z2_delta is (RxN)
            a2_delta = z2_delta*reluder(a2)           # a2_delta is (RxN)
        
            z1_delta = np.dot(a2_delta,weights[1].T)  # z1_delta is (RxN)
            a1_del = z1_delta*reluder(a1)           # a1_del is (RxN)
    
        
            w3 = weights[2] -lr * np.dot(a2.T, a3_del)+ mf * momentum[0]                  #w3 is (Nx10)
            b3 = biases[2]- lr * np.sum(a3_del, axis=0, keepdims=True)+ mf * momentum[3]  #b3 is (1x10)
        
            w2 = weights[1]- lr * np.dot(a1.T,a2_delta)+ mf * momentum[1]                   #w2 is (NxN)
            b2 = biases[1]- lr * np.sum(a2_delta, axis=0, keepdims=True)+ mf * momentum[4]  #b2 is (1x5)
        
            w1 = weights[0]- lr * np.dot(x.T, a1_del) + mf * momentum[2]                  #w1 is (784xN)
            b1 = biases[0]- lr * np.sum(a1_del, axis=0) + mf * momentum[5]                #b1 is (1xN)
            
            momentum=[]
            momentum.extend([- lr * np.dot(a2.T, a3_del),- lr * np.dot(a1.T,a2_delta),- lr * np.dot(x.T, a1_del),-lr*np.sum(a3_del, axis=0, keepdims=True),-lr*np.sum(a2_delta, axis=0, keepdims=True),-lr * np.sum(a1_del, axis=0)])
        

        weights=[]
        biases=[]
        
        weights.extend([w1,w2,w3])
        biases.extend([b1,b2,b3])
        
        return weights,biases


momentum=[0,0,0,0,0,0]   
results=NeuralNet(x_train,y_train,w,b,lr,'relu',momentum,0.5,1000)   

print('The updates weights are',results[0])
print('The updates biases are',results[1])

The updates weights are [array([[ 0.7912309 , -0.00858001, -0.83671896, ...,  0.71168652,
         0.75705463,  1.07670403],
       [ 1.28676713,  1.15841216, -2.3566597 , ...,  1.43663196,
         0.73509768, -0.95398529],
       [ 0.48931322,  0.1899332 ,  0.23887336, ..., -1.08585005,
        -0.76292394, -0.08696089],
       ...,
       [ 0.12861635, -0.37682779,  0.5718788 , ..., -0.33556216,
        -0.48094508, -0.34561233],
       [ 0.29602554,  0.0513511 , -0.37836077, ...,  0.96476086,
        -0.86233459, -1.0396994 ],
       [ 1.01914873, -0.42949699,  0.92169447, ..., -1.48373662,
        -1.0960656 , -0.55223783]]), array([[-1.44965606,  0.63931904,  1.22377172, -0.5576201 ,  0.73985256,
        -0.51181977, -0.81679159,  1.58215133,  0.80864215, -0.60910377,
        -0.38378825,  0.59580013, -3.0819262 ,  0.85642175,  0.42960341,
         0.23519655, -0.72184062,  1.01094028,  0.48839639,  0.45682418],
       [ 0.26899721,  1.08562868,  0.30556526, -0.42594275,  0.83583